# Initial AI Agent PPO

In [ ]:
# to read later: https://raw.githubusercontent.com/ray-project/ray/master/rllib/examples/unity3d_env_local.py


from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium
import ray
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.sac import SACConfig
from ray import tune
import json
import numpy as np

ray.shutdown()

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 1024
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2


def env_creator(env_config):
  env = gymnasium.make("real-time-gym-v1", config=my_config)
  return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", env_creator)

ray.init()

#agent = ppo.PPO(env="gt-rtgym-env-v1", config=ppoconfig)

algo = (
    PPOConfig()
    #SACConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        #batch_mode="truncate_episodes",
        #rollout_fragment_length=128
        )
    .framework("torch")
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
            #lr=0.0003,
            #lambda_=0.95,
            #gamma=0.99,
            sgd_minibatch_size=128,
            train_batch_size=1024,
            #num_sgd_iter=8,
            #clip_param=0.2,
            model={"fcnet_hiddens": [12, 12]},
        )
    .build()
)

N = 500
results = []
episode_data = []
episode_json = []


for n in range(N):
    result = algo.train()
    results.append(result)
  
    episode = {
        "n": n,
        "episode_reward_mean": result["episode_reward_mean"], 
        "episode_reward_max":  result["episode_reward_max"],  
        "episode_len_mean":    result["episode_len_mean"],
    }

    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    
    print('Loop: ', n, 'Max reward: ', {episode["episode_reward_max"]})
    
path_to_checkpoint = algo.save()
print(
    "An Algorithm checkpoint has been created inside directory: "
    f"'{path_to_checkpoint}'."
)

ray.shutdown()

In [ ]:
from ray.rllib.algorithms.algorithm import Algorithm
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium
import ray
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.sac import SACConfig
import json
import numpy as np

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 256
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

def env_creator(env_config):
  env = gymnasium.make("real-time-gym-v1", config=my_config)
  return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", env_creator)

# Use the Algorithm's `from_checkpoint` utility to get a new algo instance
# that has the exact same state as the old one, from which the checkpoint was
# created in the first place:
loadedAlgo = Algorithm.from_checkpoint(path_to_checkpoint)

In [ ]:

N = 300
results = []
episode_data = []
episode_json = []


for n in range(N):
    result = loadedAlgo.train()
    results.append(result)
  
    episode = {
        "n": n,
        "episode_reward_mean": result["episode_reward_mean"], 
        "episode_reward_max":  result["episode_reward_max"],  
        "episode_len_mean":    result["episode_len_mean"],
    }

    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    
    print('Loop: ', n, 'Max reward: ', {episode["episode_reward_max"]})

In [ ]:
path_to_checkpoint = algo.save()
print(
    "An Algorithm checkpoint has been created inside directory: "
    f"'{path_to_checkpoint}'."
)

In [ ]:
ray.shutdown()

# SAC Current Developing

In [2]:
# to read later: https://raw.githubusercontent.com/ray-project/ray/master/rllib/examples/unity3d_env_local.py
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium
import ray
from ray import tune, air
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.sac import SACConfig
from ray.rllib.algorithms.a3c import A3CConfig
import json
import numpy as np
from time import sleep
ray.shutdown()

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 128
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

def env_creator(env_config):
  env = gymnasium.make("real-time-gym-v1", config=my_config)
  return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", env_creator)
ray.init()

algo = (
    #PPOConfig()
    SACConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        # batch_mode="truncate_episodes",
        # rollout_fragment_length=128,
        )
    .framework("torch")
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
            #lr=tune.grid_search([0.01, 0.001, 0.0001])
            #lambda_=0.95,
            #gamma=0.99,
            #sgd_minibatch_size=128,
            #train_batch_size=1024,
            #num_sgd_iter=8,
            #clip_param=0.2,
            #model={"fcnet_hiddens": [12, 12]},
        )
    .build()
)

N = 50

for n in range(N):
    result = algo.train()
    if n % 5 == 0:
        print("Saved", n)
        algo.save()
path_to_checkpoint = algo.save()
print(
    "An Algorithm checkpoint has been created inside directory: "
    f"'{path_to_checkpoint}'."
)

ray.shutdown()

2023-05-08 23:34:47,275	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(RolloutWorker pid=10332) GT Real Time instantiated
(RolloutWorker pid=10332) GT AI Server instantiated for rtgym
(RolloutWorker pid=10332) starting up on localhost port 9999
(RolloutWorker pid=10332) Waiting for a connection
(RolloutWorker pid=10332) Connection from ('127.0.0.1', 49279)
(RolloutWorker pid=10332) default SAC action


2023-05-08 23:35:02,844	INFO trainable.py:172 -- Trainable.setup took 13.980 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-05-08 23:35:02,846	WARNING util.py:67 -- Install gputil for GPU system monitoring.


(RolloutWorker pid=10332) reset triggered
(RolloutWorker pid=10332) reload save


(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.16870019999999997
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


...

(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.430445600000013
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.3343477999999891
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.3332873000000234
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTA

(RolloutWorker pid=10332) reset triggered
(RolloutWorker pid=10332) reload save


(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.8538044999999954
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.37082430000000954
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.36751160000000027
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\

KeyboardInterrupt: 

(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.3249324000000229
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


# A3C Method

In [3]:
# to read later: https://raw.githubusercontent.com/ray-project/ray/master/rllib/examples/unity3d_env_local.py
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium
import ray
from ray import tune, air
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.sac import SACConfig
from ray.rllib.algorithms.a3c import A3CConfig
import json
import numpy as np
from time import sleep
ray.shutdown()

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 256
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

def env_creator(env_config):
  env = gymnasium.make("real-time-gym-v1", config=my_config)
  return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", env_creator)
ray.init()

algo = (
    #PPOConfig()
    #SACConfig()
    A3CConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        # batch_mode="truncate_episodes",
        # rollout_fragment_length=128,
        )
    .framework("torch")
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
            #lr=tune.grid_search([0.01, 0.001, 0.0001])
            #lambda_=0.95,
            #gamma=0.99,
            #sgd_minibatch_size=128,
            #train_batch_size=1024,
            #num_sgd_iter=8,
            #clip_param=0.2,
            #model={"fcnet_hiddens": [12, 12]},
        )
    .build()
)

N = 200

for n in range(N):
    result = algo.train()
    if n % 5 == 0:
        print("Saved", n)
        algo.save()
path_to_checkpoint = algo.save()
print(
    "An Algorithm checkpoint has been created inside directory: "
    f"'{path_to_checkpoint}'."
)

ray.shutdown()

2023-05-08 23:39:53,959	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(RolloutWorker pid=13724) GT Real Time instantiated
(RolloutWorker pid=13724) GT AI Server instantiated for rtgym
(RolloutWorker pid=13724) starting up on localhost port 9999
(RolloutWorker pid=13724) Waiting for a connection


2023-05-08 23:40:02,079	WARNING util.py:67 -- Install gputil for GPU system monitoring.


(RolloutWorker pid=13724) Connection from ('127.0.0.1', 49928)
(RolloutWorker pid=13724) default SAC action
(RolloutWorker pid=13724) reset triggered
(RolloutWorker pid=13724) reload save
Saved 0
(RolloutWorker pid=13724) reset triggered
(RolloutWorker pid=13724) reload save


(RolloutWorker pid=13724) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.14768359999998637
(RolloutWorker pid=13724)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


...

Saved 45
(RolloutWorker pid=13724) reset triggered
(RolloutWorker pid=13724) reload save


(RolloutWorker pid=13724) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.15976909999710642
(RolloutWorker pid=13724)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=13724) reset triggered
(RolloutWorker pid=13724) reload save


(RolloutWorker pid=13724) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.13489609999709273
(RolloutWorker pid=13724)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


An Algorithm checkpoint has been created inside directory: 'C:\Users\nadir/ray_results\A3C_gt-rtgym-env-v1_2023-05-08_23-39-55p4qfi8ak\checkpoint_000050'.


In [ ]:
# to read later: https://raw.githubusercontent.com/ray-project/ray/master/rllib/examples/unity3d_env_local.py
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium
import ray
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.algorithm import Algorithm
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.sac import SACConfig
from ray.rllib.algorithms.a3c import A3CConfig
import json
import numpy as np
from time import sleep

ray.shutdown()

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
#my_config["ep_max_length"] = 512
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2


def env_creator(env_config):
  env = gymnasium.make("real-time-gym-v1", config=my_config)
  return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", env_creator)

loadedAlgo = Algorithm.from_checkpoint(path_to_checkpoint)

loadedAlgo.train()

In [ ]:
import ray
from ray import air, tune
ray.shutdown()
ray.init()


config = PPOConfig().training(lr=tune.grid_search([0.01, 0.001, 0.0001])).environment(env="CartPole-v1")

tuner = tune.Tuner(
    "PPO",
    run_config=air.RunConfig(
        stop={"episode_reward_mean": 150},
    ),
    param_space=config,
)

tuner.fit()